<a href="https://colab.research.google.com/github/Bogula/GPT/blob/main/Hella.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai -q
!pip install langchain -q
!pip install chromadb -q
!pip install tiktoken -q
!pip install pypdf -q

In [ ]:
import pandas as pd
import os
os.environ["OPENAI_API_KEY"] = "YOUR KEY"

In [ ]:
df = pd.read_csv('tickets.csv', sep=";")

In [ ]:
df

In [ ]:
#Simple GPT3 Use
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

response = openai.Completion.create(
  model="text-davinci-003",
  prompt="Welches Bauteil könnte bei folgendem Text potenziell defekt sein?\n\nHallo dieses fahrzeug hat im 1 und 2 Gang keine Leistung. Dann ab ca. 3000 Umdrehungen ist die Leistung da. Keine Fehler hinterlegt.Ladedruck Sollwert 1000mbar Ist wert ca.400mbar wenn keine leistung da ist.Wenn Leistung da ist ca. 800mbar. Turboladerwelle leichtgänging und ohne spiel.Turbolader gestänge auch leichtgänig. mfg.\n\n\n\n",
  temperature=0.31,
  max_tokens=256,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
print(response)

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nDer Turbolader k\u00f6nnte defekt sein."
    }
  ],
  "created": 1683808763,
  "id": "cmpl-7EzrnsrTUPDVjRRqaqlqTONrDEQHR",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 160,
    "total_tokens": 176
  }
}


In [ ]:
#Langchain als Knowledge-Base
from langchain.document_loaders import TextLoader

loader = TextLoader("tickets.csv")


In [ ]:
#Erstellen des Vectorspaces
from langchain.indexes import VectorstoreIndexCreator
index = VectorstoreIndexCreator().from_loaders([loader])

In [ ]:
#Query gegen die Knowledge Base
query = "problem ist: FC p0335 KW Sensor Signal fehlerhaft,  FC p0100 LMM Funktion fehlerhaft, FC p0340 NW Sensor Signal fehlerhaft. Antworte auf Deutsch"
query = "Problem bei Seicento Benziner ist: springt manchmal nicht an. canbus funktion fehlerhaft, codeanforderung nicht vorhanden aus der WFS. Was kann eine Lösung sein. Eventuelle Alternative Lösung? Antworte auf Deutsch"
index.query_with_sources(query)

{'question': 'Problem bei Seicento Benziner ist: springt manchmal nicht an. canbus funktion fehlerhaft, codeanforderung nicht vorhanden aus der WFS. Was kann eine Lösung sein. Eventuelle Alternative Lösung? Antworte auf Deutsch',
 'answer': ' Eine mögliche Lösung für das Problem beim Seicento Benziner könnte der Kurbelwellensensor sein. Eine alternative Lösung könnte der Nockenwellensensor sein.\n',
 'sources': 'data/tickets.csv'}

Split docs
```



Use Semantic Search with CSV, line by line


In [ ]:
!pip install unstructured
!pip install pdf2image
!pip install pytesseract

In [ ]:
# Data Ingestion

from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='/content/data/tickets.csv', csv_args={'delimiter': ';'})

#data = loader.load()


#for line in data:
# print(line)

documents = loader.load()


In [ ]:
print(f"Total number of documents: {len(documents)}")

Total number of documents: 1000


In [ ]:
#Use Langchain Retrieval with Chat
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.chains import ConversationalRetrievalChain

In [ ]:
from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(temperature=0,model_name="gpt-3.5-turbo")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[";"," ", ",", "\n"])


documents = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()

vectorstore = Chroma.from_documents(documents, embeddings)

# Initialise Langchain - Conversation Retrieval Chain
qa = ConversationalRetrievalChain.from_llm(ChatOpenAI(temperature=0), vectorstore.as_retriever())



In [ ]:
import textwrap
user_message="What is important if I have to terminate a contract?"
history = []
chat_history = []

while True:
    user_message = input("Problem: ")

    if user_message == "stop":
        break
    response = qa({"question": user_message + "Was könnte die Lösung sein, auf Deutsch", "chat_history": history})
    history.append((user_message, response["answer"]))
    print(textwrap.fill(response["answer"], width=80))
    print("")

In [ ]:
#Einfache semantische Suche ohne Chat mit Rückgabe der Antwortkandidaten
docsearch = Chroma.from_documents(documents, embeddings, metadatas=[{"source": str(i)} for i in range(len(documents))])

In [ ]:
query = "Fiat Seicento, NW aus, Sensor keine Daten"
docs = docsearch.similarity_search(query)

In [ ]:
docs

[Document(page_content='\ufeffmanufacturer: Fiat\nmodelseries: Seicento (187)\nmodeltyp: Seicento 1.1i Sporting\nshort_name: 187\nengine_code: 187 A1.000\nfuel_type: B\ncomponent_name: Kurbelwellensensor\nproblem: Schlüssellampe für WFS ist beim Start aus----------------------\nfzg springt sporadisch nicht an-\nFC: 4 systemspannung spannung stromversorgung fehlerhaft , 2 schlüssel funktion fehlerhaft- nicht vorhanden , 3 canbus funktion fehlerhaft, codeanforderung nicht vorhanden- aus der WFS\nsolution: KW Sensor und Kraftstoffdruck', metadata={'source': '/content/data/tickets.csv', 'row': 33}),
 Document(page_content='\ufeffmanufacturer: Fiat\nmodelseries: Ducato-V (250)\nmodeltyp: Ducato 100 2.2 16V JTD\nshort_name: 250\nengine_code: 4HV (P22DTE)\nfuel_type: D\ncomponent_name: Nockenwellensensor\nproblem: .. Bitte nochmal RR..\nFZ fährt gut aber plötzlich knattert es als wenn der Motor kaputt gehen würde, \nMotorkontrolleuchten gehen an\naus motor, \nFC P 0401 FC P 0641, FC P 0103, F

In [ ]:
for i in range(len(docs)):
  print (docs[i])

page_content='\ufeffmanufacturer: Audi\nmodelseries: A6\nmodeltyp: A6 2.4i V6 30V\nshort_name: 4F\nengine_code: BDW\nfuel_type: B\ncomponent_name: Nockenwellensensor\nproblem: FC P0341 NW-Sensor Signal fehlerhaft,  Fehler ist permanent,  \nFC P0172 Gemisch Bank 1 Gemisch zu fett,  aus Motorstg. \nNW-Sensor bereits erneuert,   Mil geht an,\nsolution: Kein Kettenrasseln \nNWsensor neu \nhatte auch schon verbrennungsaussetzer aus 1,2,3\nNW sensor und Verstellung prüfen \nKat????' metadata={'source': '/content/data/tickets.csv', 'row': 627}
page_content='\ufeffmanufacturer: Audi\nmodelseries: A4\nmodeltyp: A4 1.8i 16V TFSI Avant\nshort_name: 8K\nengine_code: CDHB\nfuel_type: B\ncomponent_name: Nockenwellensensor\nproblem: Fahrzeug hat eine neue steuerkette bekommen und der zylinderkopf wurde überholt neue ventile etc. \n5531 NOCKENWELLENSENSOR 1 / SIGNAL ZU HOCH\n2980 Variables Saugrohr Bank 1 Rückmeldung\nsolution: Verkabelung NWsensor prüfen \nSaugrohrklappen GE durchführen \nEmail SLP M

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.llms import OpenAI
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
query = "Was könnte bei folgendem Auto Audi A3, mit dem Problem Hallsensor und Auto geht aus die Lösung sein. Antwort auf Deutsch"
#query = "Problem: FZG  im 1 und 2 Gang keine Leistung. Dann ab ca. 3000 Umdrehungen ist die Leistung da. Keine Fehler hinterlegt.Ladedruck Sollwert 1000mbar Ist wert ca.400mbar wenn keine leistung da ist.Wenn Leistung da ist ca. 800mbar. Turboladerwelle leichtgänging und ohne spiel.Turbolader gestänge auch leichtgänig. Was könnte die Lösung sein? query = " canbus funktion fehlerhaft, codeanforderung nicht vorhanden- aus der WFS"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' Bei dem Auto Audi A3 könnte die Lösung für das Problem Hallsensor und Auto geht aus der Austausch des Hallsensors sein.\nSOURCES: /content/data/tickets.csv'}

In [ ]:
#Probability der Kandidaten für das nächste Token
import openai

# Set up your OpenAI API credentials
openai.api_key = os.getenv("OPENAI_API_KEY")

# Define your input text
input_text = 'Once upon a time'

# Generate the probability distribution
response = openai.Completion.create(
  engine='text-davinci-003',  # Specify the GPT-3 model variant
  prompt=input_text,
  temperature=0.0,  # Set temperature to 0 for deterministic output
  max_tokens=1,  # Predict a single token
  n=1,  # Generate a single completion
  stop=None,  # Continue generating until max_tokens limit is reached
  logprobs=5,  # Request log probabilities
)

# Extract the log probabilities and convert them to a probability distribution
logprobs = response.choices[0].logprobs

print(logprobs)

#probabilities = [lp for lp in logprobs.token_logprobs[0]]

# Display the predicted words and their probabilities
#predicted_words = logprobs.tokens[1:]
#for word, prob in zip(predicted_words, probabilities):
#    print(f'Word: {word}, Probability: {prob}')

{
  "text_offset": [
    16
  ],
  "token_logprobs": [
    -0.9235233
  ],
  "tokens": [
    " there"
  ],
  "top_logprobs": [
    {
      "\n": -1.1671022,
      " a": -5.8914065,
      " in": -4.347084,
      " there": -0.9235233,
      ",": -1.3568397
    }
  ]
}
